## I Filter

In [1]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/apassGD421.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalI_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_I': catalog_match['Ic'],
                    'KnownI_err': catalog_match['Ic_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"IResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")





the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i/IResult_2.15076185278.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i/IResult_2.16714233109.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i/IResult_1.66239524237.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i/IResult_1.71013678925.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i/IResult_1.88574519777.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i/IResult_2.45572372756.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_i/IResult_1.91706279757.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


## R Filter

In [5]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/apassGD421.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_R': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalR_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_R': catalog_match['Rc'],
                    'KnownR_err': catalog_match['Rc_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"RResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_2.13867993725.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_1.89315380946.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_1.6436992097.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_2.4236963029.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_1.86208945872.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_2.12207091126.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_1.6715689965.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_r/RResult_1.69066079001.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


## B Filter

In [3]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/apassGD421.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_b"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalB_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_B': catalog_match['B'],
                    'KnownB_err': catalog_match['B_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"BResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_b/BResult_1.60729773827.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_b/BResult_2.11052755732.csv
Failed to process calibrated_GD 421-20240318at040540_-20-3-300-B.fts: Number of world inputs (1) does not match expected (2)
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_b/BResult_2.06610890535.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_b/BResult_1.86957829916.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_b/BResult_2.391752113.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


## V Filter

In [4]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/apassGD421.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalV_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_V': catalog_match['V'],
                    'KnownV_err': catalog_match['V_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"VResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v/VResult_1.62527902818.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v/VResult_2.08229358105.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v/VResult_2.35944558709.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v/VResult_2.09389071635.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v/VResult_1.84592332402.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v/VResult_1.8388588425.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/GD 421/calibrated_v/VResult_1.65244711489.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
